Unkorellierte Beobachtungen einlesen

In [1]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt

Aufgabe 1:

In [29]:
input1 = np.loadtxt("ue01_bsp1.txt")
print(input1)

[[ 1.   2.9  4.   5.   7.2  8.7]
 [ 2.   3.   3.7  5.2  7.2  9.2]
 [ 3.   3.   4.   5.   7.1  8.9]
 [ 4.   3.   3.9  5.   7.1  9.1]
 [ 5.   2.9  3.9  5.   7.3  9. ]
 [ 6.   2.9  3.9  4.9  7.2  8.6]
 [ 7.   3.1  4.   5.1  7.2  9.2]
 [ 8.   2.9  4.   5.   6.9  9. ]
 [ 9.   3.   4.   5.1  7.2  9.3]
 [10.   2.9  4.1  5.1  6.6  9.1]
 [11.   3.   4.2  5.   7.3  9. ]]


Standardabweichungen:

In [30]:
sigma_123 = 0.1
sigma_45 = 0.2

Sequentielle Ausgleichung:

Aufgabe 2:

In [31]:
input2 = np.loadtxt("ue01_bsp2.txt")
print(input2)

[[ 1.   0.6  4.2 10.   4.5 14.1]
 [ 2.   1.5  4.   8.3  5.5 12.2]
 [ 3.   1.8  3.9  6.7  5.9 10.8]
 [ 4.   2.6  4.   5.7  6.7  9.9]
 [ 5.   2.8  3.9  5.3  7.3  9.1]
 [ 6.   3.   4.   4.8  7.3  9.2]
 [ 7.   2.8  4.   5.1  7.3  9.1]
 [ 8.   2.5  3.9  5.7  6.9  9.9]
 [ 9.   2.2  3.9  6.7  6.1 10.7]
 [10.   1.3  4.   8.1  5.4 11.9]
 [11.   0.6  3.9 10.2  4.6 13.9]]
